# ADARUL

In [1]:
import rul_datasets
import rul_adapt
import pytorch_lightning as pl
import omegaconf

## Reproduce original configurations

You can reproduce the original experiments by Ragab et al. by using the `get_adarul` constructor function.
Known differences to the original paper are:

* the target data is truncated with 80% `percent_broken` to exclude data from the point of failure

Additional kwargs for the trainer, e.g. accelerator="gpu" for training on a GPU, can be passed to the function as a dictionary.
The first dictionary is used for the pre-training trainer and the second one for the main trainer.

In [2]:
pl.seed_everything(42, workers=True)  # makes is reproducible
pre_training, main_training = rul_adapt.construct.get_adarul(
    3, 1, {"max_epochs": 1}, {"max_epochs": 1}
)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


The function returns two tuples.
The first contains everything needed for pre-training, the second everything needed for the main training.

In [3]:
pre_dm, pre_approach, pre_trainer = pre_training
pre_trainer.fit(pre_approach, pre_dm)

Missing logger folder: /home/tilman/Programming/rul-adapt/examples/lightning_logs

  | Name               | Type               | Params
----------------------------------------------------------
0 | train_loss         | MeanSquaredError   | 0     
1 | val_loss           | MeanSquaredError   | 0     
2 | _feature_extractor | LstmExtractor      | 62.5 K
3 | _regressor         | FullyConnectedHead | 6.3 K 
----------------------------------------------------------
68.7 K    Trainable params
0         Non-trainable params
68.7 K    Total params
0.275     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


After pre-training, we can use the pre-trained networks to initialize the main training.
The networks of the pre-training approach, i.e. `feature_extractor` and `regressor`, can be accessed as properties.

In [4]:
dm, approach, domain_disc, trainer = main_training
approach.set_model(pre_approach.feature_extractor, pre_approach.regressor, domain_disc)
trainer.fit(approach, dm)
trainer.test(approach, dm)


   | Name               | Type               | Params
-----------------------------------------------------------
0  | gan_loss           | BCEWithLogitsLoss  | 0     
1  | val_source_rmse    | MeanSquaredError   | 0     
2  | val_target_rmse    | MeanSquaredError   | 0     
3  | val_source_score   | RULScore           | 0     
4  | val_target_score   | RULScore           | 0     
5  | test_source_rmse   | MeanSquaredError   | 0     
6  | test_target_rmse   | MeanSquaredError   | 0     
7  | test_source_score  | RULScore           | 0     
8  | test_target_score  | RULScore           | 0     
9  | _feature_extractor | LstmExtractor      | 62.5 K
10 | _regressor         | FullyConnectedHead | 6.3 K 
11 | _domain_disc       | FullyConnectedHead | 6.3 K 
-----------------------------------------------------------
75.0 K    Trainable params
0         Non-trainable params
75.0 K    Total params
0.300     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test/source_rmse         81.89234924316406
    test/source_score           333703.4375
    test/target_rmse                                  73.20379638671875
    test/target_score                                   123315.53125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/source_rmse/dataloader_idx_0': 81.89234924316406,
  'test/source_score/dataloader_idx_0': 333703.4375},
 {'test/target_rmse/dataloader_idx_1': 73.20379638671875,
  'test/target_score/dataloader_idx_1': 123315.53125}]

If you only want to see the hyperparameters, you can use the `get_adarul_config` function.
This returns an `omegaconf.DictConfig` which you can modify.
Afterwards, you can pass the config to `adarul_from_config` to receive the training-ready approach.

In [6]:
three2one_config = rul_adapt.construct.get_adarul_config(3, 1)
print(omegaconf.OmegaConf.to_yaml(three2one_config, resolve=True))

dm:
  source:
    _target_: rul_datasets.CmapssReader
    fd: 3
    window_size: 30
  target:
    fd: 1
    percent_broken: 0.8
  batch_size: 10
feature_extractor:
  _target_: rul_adapt.model.LstmExtractor
  input_channels: 14
  lstm_units:
  - 32
  - 32
  - 32
  bidirectional: true
regressor:
  _target_: rul_adapt.model.FullyConnectedHead
  input_channels: 64
  act_func_on_last_layer: false
  units:
  - 64
  - 32
  - 1
  dropout: 0.5
domain_disc:
  _target_: rul_adapt.model.FullyConnectedHead
  input_channels: 64
  act_func_on_last_layer: false
  units:
  - 64
  - 32
  - 1
adarul_pre:
  _target_: rul_adapt.approach.AdaRulApproachPretraining
  lr: 0.0001
adarul:
  _target_: rul_adapt.approach.AdaRulApproach
  lr: 0.0001
trainer_pre:
  _target_: pytorch_lightning.Trainer
  max_epochs: 100
trainer:
  _target_: pytorch_lightning.Trainer
  max_epochs: 20



## Run your own experiments

You can use the ADARUL implementation to run your own experiments with different hyperparameters or on different datasets.
Here we build an approach with an CNN feature extractor.


In [9]:
source = rul_datasets.CmapssReader(3)
target = source.get_compatible(1, percent_broken=0.8)

pre_dm = rul_datasets.RulDataModule(source, batch_size=32)
dm = rul_datasets.DomainAdaptionDataModule(
    pre_dm, rul_datasets.RulDataModule(target, batch_size=32),
)

feature_extractor = rul_adapt.model.CnnExtractor(
    input_channels=14,
    conv_filters=[16, 16, 16],
    seq_len=30,
    fc_units=8,
)
regressor = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)
domain_disc = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)

pre_approach = rul_adapt.approach.AdaRulApproachPretraining(lr=0.001)
pre_approach.set_model(feature_extractor, regressor)
pre_trainer = pl.Trainer(max_epochs=1)
trainer.fit(pre_approach, pre_dm)

approach = rul_adapt.approach.AdaRulApproach(lr=0.001)
approach.set_model(
    pre_approach.feature_extractor, pre_approach.regressor, domain_disc
)
trainer = pl.Trainer(max_epochs=1)
trainer.fit(approach, dm)
trainer.test(approach, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name               | Type               | Params
----------------------------------------------------------
0 | train_loss         | MeanSquaredError   | 0     
1 | val_loss           | MeanSquaredError   | 0     
2 | _feature_extractor | CnnExtractor       | 5.3 K 
3 | _regressor         | FullyConnectedHead | 81    
----------------------------------------------------------
5.4 K     Trainable params
0         Non-trainable params
5.4 K     Total params
0.022     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name               | Type               | Params
-----------------------------------------------------------
0  | gan_loss           | BCEWithLogitsLoss  | 0     
1  | val_source_rmse    | MeanSquaredError   | 0     
2  | val_target_rmse    | MeanSquaredError   | 0     
3  | val_source_score   | RULScore           | 0     
4  | val_target_score   | RULScore           | 0     
5  | test_source_rmse   | MeanSquaredError   | 0     
6  | test_target_rmse   | MeanSquaredError   | 0     
7  | test_source_score  | RULScore           | 0     
8  | test_target_score  | RULScore           | 0     
9  | _feature_extractor | CnnExtractor       | 5.3 K 
10 | _regressor         | FullyConnectedHead | 81    
11 | _domain_disc       | FullyConnectedHead | 81    
-------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test/source_rmse         83.66500091552734
    test/source_score           374342.0625
    test/target_rmse                                  84.70077514648438
    test/target_score                                    344921.375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/source_rmse/dataloader_idx_0': 83.66500091552734,
  'test/source_score/dataloader_idx_0': 374342.0625},
 {'test/target_rmse/dataloader_idx_1': 84.70077514648438,
  'test/target_score/dataloader_idx_1': 344921.375}]